# IMDb Top 250 Movies Scraper

This project scrapes data from IMDb's "Top 250 Movies" chart, preprocesses and cleans the data, and saves it into structured CSV files.

## Step 1: Scrape Data
This step fetches the raw data from IMDb and saves it as a JSON file.

In [26]:

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# IMDb Top 250 movies URL
url = 'https://www.imdb.com/chart/top/'

# Headers to mimic a real browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Send an HTTP GET request to fetch the page content
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the page!")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    exit()

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the JSON-LD data containing movie details
script_tag = soup.find('script', type='application/ld+json')
if script_tag:
    data = json.loads(script_tag.string)
else:
    print("No JSON data found.")
    exit()

# Save raw data
with open("raw_imdb_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

print("Raw data saved as 'raw_imdb_data.json'")


Successfully fetched the page!
Raw data saved as 'raw_imdb_data.json'


## Step 2: Preprocess Data
This step processes the raw data and saves it as a CSV file.

In [27]:

# Load raw JSON data
with open("raw_imdb_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Initialize lists to store movie details
titles, urls, descriptions, ratings = [], [], [], []

# Extract movie details
if 'itemListElement' in data:
    for item in data['itemListElement']:
        movie = item['item']
        titles.append(movie.get('name', 'N/A'))
        urls.append(movie.get('url', 'N/A'))
        descriptions.append(movie.get('description', 'N/A'))
        ratings.append(movie.get('aggregateRating', {}).get('ratingValue', 'N/A'))

# Create a DataFrame
df_sample = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Description': descriptions,
    'Rating': ratings
})

# Save a sample of the data
df_sample.head(10).to_csv("initial_sample.csv", index=False)

print("Initial data sample saved as 'initial_sample.csv'")


Initial data sample saved as 'initial_sample.csv'


## Step 3: Data Cleaning and Preprocessing
This step involves handling missing values, converting ratings to numeric, and cleaning text fields for the sample dataset.


In [28]:
#step-3 developing the data preprocessing script
import pandas as pd

# Load the initial sample dataset
df_sample = pd.read_csv("initial_sample.csv")

# Preprocessing the Sample Data

# 1. Handle missing values (NaN to 'N/A')
df_sample.fillna('N/A', inplace=True)

# 2. Convert 'Rating' to numeric values, errors will be converted to NaN
df_sample['Rating'] = pd.to_numeric(df_sample['Rating'], errors='coerce')


# 4. Normalize text fields (strip leading/trailing spaces)
df_sample['Title'] = df_sample['Title'].str.strip()
df_sample['Description'] = df_sample['Description'].str.strip()

# Save the preprocessed data
df_sample.to_csv("preprocessed_sample.csv", index=False)

print("Preprocessed sample data saved as 'preprocessed_sample.csv'")


Preprocessed sample data saved as 'preprocessed_sample.csv'


## Step 4: Handle Missing or Invalid Data
This step deals with handling missing values, duplicates, outliers in ratings, and ensures the data is clean for analysis.


In [29]:


# Step 1: Load the initial sample data (generated in Step 2)
df_sample = pd.read_csv("initial_sample.csv")

# Step 2: Handle missing or invalid data
# Check for missing values before cleaning
print("Missing values before cleaning:\n", df_sample.isnull().sum())

# Fill missing values with 'N/A' for categorical columns and mean for numerical columns
df_sample['Rating'] = pd.to_numeric(df_sample['Rating'], errors='coerce')  # Coerce invalid values to NaN
df_sample['Rating'].fillna(df_sample['Rating'].mean(), inplace=True)  # Replace NaN ratings with the mean rating
df_sample['Description'].fillna('No description available', inplace=True)  # Replace missing descriptions with default text

# If any other categorical columns are missing data, fill them with a default string
df_sample['Title'].fillna('Unknown Title', inplace=True)
df_sample['URL'].fillna('No URL available', inplace=True)

# Step 3: Remove duplicates
df_sample.drop_duplicates(inplace=True)

# Step 4: Handle outliers for 'Rating' column
# Identify extreme values (ratings greater than 10 or less than 0)
df_sample['Rating'] = df_sample['Rating'].apply(lambda x: 10 if x > 10 else (0 if x < 0 else x))

# Step 5: Check cleaned data
print("Cleaned data sample:\n", df_sample.head())

# Step 6: Save the cleaned data
df_sample.to_csv("cleaned_sample_data.csv", index=False)
print("Cleaned sample data saved as 'cleaned_sample_data.csv'")


Missing values before cleaning:
 Title          0
URL            0
Description    0
Rating         0
dtype: int64
Cleaned data sample:
                       Title                                    URL  \
0  The Shawshank Redemption  https://www.imdb.com/title/tt0111161/   
1             The Godfather  https://www.imdb.com/title/tt0068646/   
2           The Dark Knight  https://www.imdb.com/title/tt0468569/   
3     The Godfather Part II  https://www.imdb.com/title/tt0071562/   
4              12 Angry Men  https://www.imdb.com/title/tt0050083/   

                                         Description  Rating  
0  A banker convicted of uxoricide forms a friend...     9.3  
1  The aging patriarch of an organized crime dyna...     9.2  
2  When a menace known as the Joker wreaks havoc ...     9.0  
3  The early life and career of Vito Corleone in ...     9.0  
4  The jury in a New York City murder trial is fr...     9.0  
Cleaned sample data saved as 'cleaned_sample_data.csv'


## Step 5: Data Transformation and Final Preprocessing
This step applies final transformations like removing duplicates, handling outliers, and saving the cleaned dataset as a CSV file.


In [30]:


# Load the raw IMDb data from the JSON file
with open("raw_imdb_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract movie details for the full dataset
titles, urls, descriptions, ratings = [], [], [], []

if 'itemListElement' in data:
    for item in data['itemListElement']:
        movie = item['item']
        titles.append(movie.get('name', 'N/A'))
        urls.append(movie.get('url', 'N/A'))
        descriptions.append(movie.get('description', 'N/A'))
        ratings.append(movie.get('aggregateRating', {}).get('ratingValue', 'N/A'))

# Create a DataFrame for the full dataset
df_full = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Description': descriptions,
    'Rating': ratings
})

# Step 1: Handle missing values (NaN to 'N/A')
df_full.fillna('N/A', inplace=True)

# Step 2: Convert 'Rating' to numeric values, errors will be converted to NaN
df_full['Rating'] = pd.to_numeric(df_full['Rating'], errors='coerce')

# Step 3: Fill NaN values in 'Rating' with the mean of the 'Rating' column
df_full['Rating'].fillna(df_full['Rating'].mean(), inplace=True)

# Step 4: Normalize text fields (strip leading/trailing spaces)
df_full['Title'] = df_full['Title'].str.strip()
df_full['Description'] = df_full['Description'].str.strip()

# Step 5: Remove duplicates
df_full.drop_duplicates(inplace=True)

# Step 6: Handle outliers for 'Rating' column
# Ratings should be between 0 and 10
df_full['Rating'] = df_full['Rating'].apply(lambda x: 10 if x > 10 else (0 if x < 0 else x))

# Step 7: Check cleaned data
print("Cleaned full dataset sample:\n", df_full.head())

# Step 8: Save the cleaned full dataset
df_full.to_csv("cleaned_full_dataset.csv", index=False)
print("Cleaned full dataset saved as 'cleaned_full_dataset.csv'")


Cleaned full dataset sample:
                       Title                                    URL  \
0  The Shawshank Redemption  https://www.imdb.com/title/tt0111161/   
1             The Godfather  https://www.imdb.com/title/tt0068646/   
2           The Dark Knight  https://www.imdb.com/title/tt0468569/   
3     The Godfather Part II  https://www.imdb.com/title/tt0071562/   
4              12 Angry Men  https://www.imdb.com/title/tt0050083/   

                                         Description  Rating  
0  A banker convicted of uxoricide forms a friend...     9.3  
1  The aging patriarch of an organized crime dyna...     9.2  
2  When a menace known as the Joker wreaks havoc ...     9.0  
3  The early life and career of Vito Corleone in ...     9.0  
4  The jury in a New York City murder trial is fr...     9.0  
Cleaned full dataset saved as 'cleaned_full_dataset.csv'


## Step 6: Final Review and Save
This step applies any remaining final adjustments and saves the fully cleaned dataset.

In [33]:

# Load the cleaned full dataset (from Step 6)
df_full = pd.read_csv("cleaned_full_dataset.csv")

# Final Preprocessing
# 1. Check for and handle any remaining missing values
df_full.fillna('N/A', inplace=True)

# 2. Ensures 'Rating' is still numeric and handle any NaN values
df_full['Rating'] = pd.to_numeric(df_full['Rating'], errors='coerce')
df_full['Rating'].fillna('N/A', inplace=True)

# 3. Clean any remaining text fields (strip leading/trailing spaces)
df_full['Title'] = df_full['Title'].str.strip()
df_full['Description'] = df_full['Description'].str.strip()

#filtering out any movies with "N/A" ratings or invalid data
df_full = df_full[df_full['Rating'] != 'N/A']

# Categorizing ratings into High, Medium, and Low
df_full['Rating_Category'] = df_full['Rating'].apply(lambda x: 'High' if x >= 8 else ('Medium' if x >= 5 else 'Low'))

# Save the final dataset
df_full.to_csv("final_cleaned_dataset.csv", index=False)

print("Final cleaned dataset saved as 'final_cleaned_dataset.csv'")



Final cleaned dataset saved as 'final_cleaned_dataset.csv'


In [32]:
print(df_sample.columns)


Index(['Title', 'URL', 'Description', 'Rating'], dtype='object')
